# Section 1. Preparing the Data

### 1.1. Loading required passages (TensorFlow, NumPy, Pandas, CV2)

Importing needed packages.

**Essential Note:**
If you would like to test _Keras Facenet Feature Extraction_ which I use here, you should use older versions of _Keras_, _TensorFlow_, and probably _Python_ as well. 

**My configuration of kernel for Keras Facenet:**
- TensorFlow: 1.14.0
- Keras: 2.3.1
- Python: 3.6.2

_P.S. That was quite painful to find this configuration and avoid all the conflicts :D_

In [ ]:
# Disable warnings
import warnings
warnings.filterwarnings("ignore")

# Import pyplot from matplotlib
import matplotlib.pyplot as plt

# Import TensorFlow image preprocessing 
from tensorflow.keras.preprocessing import image as image_preprocessing
import tensorflow.keras.backend as tf_backend 
tf_backend.set_image_data_format('channels_last')

# Importing fetch_lfw_people dataset from the sklearn.datasets package
from sklearn.datasets import fetch_lfw_people

# Import all other needed packages
import os
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import face_recognition
import glob
from PIL import Image

# Internal helper imports
from helpers import graphics as help_graph
from helpers import binary_converter as help_binary
from helpers import accuracy_calculator as help_calc
from helpers import shaper as help_shaper
from helpers import vector_extractor as help_extract
from helpers import segmenter as help_segmenter

%matplotlib inline

### 1.2. Loading images dataset for checking the accuracy

Note, that if you are trying to run this notebook, run only one block from either **Option 1** or **Option 2**.

**Option 1.** Importing `fetch_lfw_people` dataset from `sklearn.datasets`

In [ ]:
# Importing dataset
lfw_people_dataset = fetch_lfw_people(min_faces_per_person=80, color=True)

# Settings images
people_images = lfw_people_dataset.images
# Setting labels
people_labels = lfw_people_dataset.target

**Option 2**. Importing `lfw` [dataset](http://vis-www.cs.umass.edu/lfw/) from the folder

In [ ]:
# Specifying path to the images
lfw_dataset_path = './Images/lfw'
# Getting array of subpaths
lfw_people_subpaths = [subdirectory[0] for subdirectory in os.walk(lfw_dataset_path)]
# Setting minimal amount of faces needed for one person
faces_per_person = 60

people_images, people_labels = [], []

# Going along the array
for person_id, subpath in enumerate(lfw_people_subpaths[1:]):
    subpath_images = [cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB) for file in glob.glob(subpath + '/*.jpg')]
    if len(subpath_images) < faces_per_person:
        continue

    for image in subpath_images[:faces_per_person]:
        people_images.append(image)
    
    people_labels += [person_id] * faces_per_person
    
# Checking if loading was completed
print(np.shape(people_images))

Converting `lfw_people` and `lfw_people_labels` to the np array 

In [ ]:
people_images_np = np.array(people_images, dtype = float)
print('Shape of images dataset:', people_images_np.shape)

Displaying the dataset to check whether we initialized everything properly

In [ ]:
gallery_offset = 0 # Can be changed to "move" the gallery
gallery_width = 3 # Can be changed to resize gallery

help_graph.display_images(
    people_images_np[(gallery_offset):(gallery_width**2 + gallery_offset)], 
    people_labels[(gallery_offset):(gallery_width**2 + gallery_offset)])

### 1.3. Forming batches of images of the same person

In [ ]:
person_id = 0 # Person's ID to be displayed
batch = help_shaper.form_batch_by_id(people_images_np, people_labels, image_id=person_id) # Getting images of a person

print('Batch shape', batch.shape) # Check whether the shape is correct

# Displaying a gallery
help_graph.display_images(
    batch[:(gallery_width**2)], 
    np.full((gallery_width**2), person_id, dtype=int), 
    gallery_size=3)

In [ ]:
person_id = 0 # Person's ID to be displayed
batch = help_shaper.form_batch_by_id(people_images_np, people_labels, image_id=person_id) # Getting images of a person

print('Batch shape', batch.shape) # Check whether the shape is correct

# Displaying a gallery
help_graph.display_images(
    batch[:(gallery_width**2)], 
    np.full((gallery_width**2), person_id, dtype=int), 
    gallery_size=3)

In [ ]:
# Distributing images into batches of the same id
image_batches = help_shaper.form_batches_list(people_images_np, people_labels)
# Id of a batch to display in the gallery
image_batch_display_id = 3

# Trying to use form_batches_numpy
image_batches_numpy = help_shaper.form_batches_numpy(people_images_np, people_labels, dataset='sklearn_lfw_people')

# Displaying a gallery
help_graph.display_images(
    image_batches_numpy[image_batch_display_id][:(gallery_width**2)], 
    np.full((gallery_width**2), image_batch_display_id, dtype=int), 
    gallery_size=3)

# Printing some info to verify that everything was implemented properly
print('Number of batches:', len(image_batches))
print('Shape of selected batch:', image_batches[image_batch_display_id].shape)
print('Shape of numpy list of batches:', image_batches_numpy.shape)

# Section 2. Extracting features from an image using Keras Facenet

#### Loading model's weights and model itself

In [ ]:
model = keras.models.load_model('models/facenet_keras.h5', compile=False)model = keras.models.load_model('models/facenet_keras.h5', compile=False)

Suppose we have a batch of size $n_b$ with images $I^{(1)}, I^{(2)}, \dots, I^{(n_b)}$.
Using FaceNet, we can extract a feature vector of dimension $n_f$, that is:
$$
f(I^{(j)}) = \begin{pmatrix} f(I^{(j)})_1 \\ f(I^{(j)})_2 \\ \vdots \\ f(I^{(j)})_{n_f} \end{pmatrix}
$$
To evaluate difference $d(I^{(m)}, I^{(k)})$ between images $I^{(m)}$ and $I^{(k)}$ we are going to simply take a square of the feature-vectors difference norm:
$$
d(I^{(m)}, I^{(k)}) = \| f(I^{(m)}) - f(I^{(k)}) \|^2 = \sum_{j=1}^{n_f} (f(I^{(m)})_j - f(I^{(k)})_j)^2
$$

### Section 2.1 (Optinal). Segmenting face from the picture

In [ ]:
# Segmenting batches (OPTIONAL)
image_batches_segmented = help_segmenter.segment_batches(image_batches)

In [ ]:
image_batches_segmented_np = help_shaper.normalize_batches_size(image_batches_segmented)
print('Normalized batches array shape', np.shape(image_batches_segmented_np))

help_graph.visualize_segmented(image_batches[4][3], image_batches_segmented_np[4][3], figure_size=7)
help_graph.visualize_segmented(image_batches[1][10], image_batches_segmented_np[1][10], figure_size=7)
help_graph.visualize_segmented(image_batches[2][20], image_batches_segmented_np[2][20], figure_size=7)

**Note!** This line will overwrite the previously formed array `image_batches_numpy`

In [ ]:
# Updating image_batches_numpy
image_batches_numpy = image_batches_segmented_np

### Section 2.2. Retrieving the set of features for our images

**Distance between vectors function**

In [ ]:
# Getting array of feature vectors
feature_vectors = help_extract.extract_vectors_from_batches(image_batches_numpy, model=model)

In [ ]:
n1, i1, k1, j1 = 0, 0, 0, 6
n2, i2, k2, j2 = 0, 0, 2, 15


help_graph.distance_visualize(image_batches[n1][i1], 
                   image_batches[k1][j1], 
                   help_extract.feature_vectors_distance(feature_vectors[n1][i1], feature_vectors[k1][j1]))
help_graph.distance_visualize(image_batches[n2][i2], 
                   image_batches[k2][j2],
                   help_extract.feature_vectors_distance(feature_vectors[n2][i2], feature_vectors[k2][j2]))

In [ ]:
# Read images
person_1 = np.array(Image.open('Images/Dima1.png'))
person_2 = np.array(Image.open('Images/Dima2.png'))

# Visualization of all distances
help_graph.distance_visualize(person_1, person_2, help_extract.get_distance_between_images(person_1, person_2, model), 'distance')
help_graph.distance_visualize(person_1, image_batches[2][3], help_extract.get_distance_between_images(person_1, image_batches[2][3], model), 'distance')
help_graph.distance_visualize(person_1, image_batches[3][10], help_extract.get_distance_between_images(person_1, image_batches[3][10], model), 'distance')
help_graph.distance_visualize(image_batches[1][15], 
                   image_batches[1][30], 
                   help_extract.get_distance_between_images(image_batches[1][15], image_batches[1][30], model), 'distance')

# Section 3. Estimating the accuracy

### 3.1. Forming set of pairs

In [ ]:
# Forming a list of pairs
pairs = help_shaper.form_pairs_list((len(feature_vectors), help_shaper.get_minimal_batch_size(feature_vectors)))

### 3.2. Finding the best threshold to estimate pair difference

In [ ]:
# Getting arrays of threshold and accuracies
thresholds, accuracies = help_calc.threshold_accuracies(pairs, feature_vectors, lower_edge=40.0, upper_edge=300.0, step=0.01)

In [ ]:
help_graph.draw_plot(thresholds, accuracies,
                     X_label='Threshold', Y_label='Binary accuracy', 
                     title='Binary accuracy dependence on threshold', color='red')

In [ ]:
best_threshold = help_calc.get_best_threshold(pairs, feature_vectors, lower_edge=40.0, upper_edge=200.0, step=0.1)
print('Best threshold is', best_threshold)
print('Binary accuracy is', help_calc.calculate_pairs_binary_accuracy(pairs, feature_vectors, threshold=best_threshold))

### 3.2. Defining the term "continious accuracy" of pair difference prediction

Now let us estimate roughly what is the accuracy of our algorithm.
Suppose we have $n_b$ batches, every one of which contains $n_i$ images:
$$
(I^{(1)}_1, I^{(1)}_2, \dots, I^{(1)}_{n_i}), (I^{(2)}_1, I^{(2)}_2, \dots\, I^{(2)}_{n_i}), \dots, (I^{(n_b)}_1, \dots, I^{(n_b)}_{n_i})
$$
In other words, $I^{(i)}_j$ is a $j$th image in $i$th batch.

We could probably use binary classification loss functions or F1 score, but we are interested not in the face recognition task, but on the feature vectors themselves as they should not differ significantly.

We will define the following "accuracy" function, where $\tau$ denotes certain threshold (in our case we have put $\tau = 100$):
$$
\alpha(I^{(n_1)}_{i_1}, I^{(n_2)}_{i_2}) = 
\begin{cases}
\max\{1- \left(\frac{d(I^{(n)}_{i_1}, I^{(n)}_{i_2})}{\tau}\right)^{\eta_1}, 0\} \; \text{if} \; n_1 = n_2 = n \\
\max\{1-\left(\frac{\tau}{d(I^{(n_1)}_{i_1}, I^{(n_2)}_{i_2})}\right)^{\eta_2}, 0\} \; \text{if} \; n_1 \neq n_2
\end{cases}
$$

The intuition behind this formula is rather simple: if both images are from the same person, then if the value is less than threshold, then the value interpolates on a segment $[0,1]$ (if distance is close to $0$, thus accuracy is close to $1$, if distance is close to $\tau$, then the accuracy tends to $0$). Same intuition if images are different, but we use the inverse relation to the power of $\eta$.

Suppose we chose $n_p$ pairs of images, each element of which has parameters $p^{(t)} = \{n^{(t)}_1, n^{(t)}_2, i^{(t)}_1, i^{(t)}_2\}$. Then our accuracy equals:
$$
\mathcal{A} = \sqrt{\frac{1}{n_p} \sum_{t=1}^{n_p} \alpha^2(I^{(n^{(t)}_1)}_{i^{(t)}_1}, I^{(n^{(t)}_2)}_{i^{(t)}_2})}
$$

We will select batches so that the number of image pairs from the same batch is the same as the number of pairs from the different batches.

In [ ]:
# Checking whether calculation of accuracy is good enough 

# Expected value: 1 - (100/200)^3 = 0.875 
print('If batches are different and distance=200, accuracy equals', help_calc.calculate_accuracy(False, 200))
# Expected value: 0 
print('If batches are the same and distance=200, accuracy equals', help_calc.calculate_accuracy(True, 200))
# Expected value: 0
print('If batches are the different and distance=90, accuracy equals', help_calc.calculate_accuracy(False, 90))
# Expected value: 1 - (64/100)^(3) = 0.737856 
print('If batches are the same and distance=64, accuracy equals', help_calc.calculate_accuracy(True, 64))

In [ ]:
#Visualization of accuracy between images

help_graph.distance_visualize(image_batches[0][0], 
                   image_batches[0][1], 
                   help_calc.calculate_pair_accuracy(feature_vectors, 0, 0, 0, 1, threshold=best_threshold))
help_graph.distance_visualize(image_batches[0][0], 
                   image_batches[1][0], 
                   help_calc.calculate_pair_accuracy(feature_vectors, 0, 0, 1, 0, threshold=best_threshold))

In [ ]:
accuracy = help_calc.calculate_pairs_accuracy([[0, 0, 0, 1], [0, 0, 1, 0]], feature_vectors, threshold=best_threshold)
print('Cumulative accuracy for two images above is', accuracy)

In [ ]:
# Getting accuracy
print('Total accuracy is', help_calc.calculate_pairs_accuracy(pairs, feature_vectors, threshold=best_threshold))
print('Total binary accuracy is', help_calc.calculate_pairs_binary_accuracy(pairs, feature_vectors, threshold=best_threshold))

# Saving the feature vector data to the CSV file

In [ ]:
def form_row_csv(feature_vectors, person_id, image_id):
    """
    Returns an array that corresponds to a single row in the csv file
    """
    
    row = [person_id, image_id]
    return np.append(row, feature_vectors[person_id][image_id])

def form_rows_csv(feature_vectors):
    """
    Returns an array with rows used to load the csv file
    """
    
    rows = []
    for person_id in range(len(feature_vectors)):
        for image_id in range(len(feature_vectors[person_id])):
            rows.append(form_row_for_csv(feature_vectors, person_id, image_id))
            
    return rows

def form_header_csv(vector_dimension):
    """
    Forms a header for the CSV file
    
    Input:
    vector_dimension -- integer that represents the dimensionality of the vector
    """
    
    header = ['person_id', 'image_id']
    for vector_id in range(vector_dimension):
        header.append('feature_' + str(vector_id))
        
    return header

In [ ]:
import csv

# Load the CSV file
with open('datasets/keras_facenet_feature_vectors.csv', 'w') as dataset:
    # Create a writer
    writer = csv.writer(dataset)
    
    # Write a header
    vector_dimension = len(feature_vectors[0][0])
    writer.writerow(form_header_csv(vector_dimension))
    
    # Write rows
    writer.writerows(form_rows_csv(feature_vectors))

# Section 4. Binary string formation 

Suppose we would like to form a binary string $s(I)=s_1s_2\dots s_k$ from a feature vector $\mathbf{f}(I) \in \mathbb{R}^{k}$ of some image $I$:
$$
\mathbf{f}(I) = \begin{bmatrix} f(I)_1 \\ f(I)_2 \\ \dots \\ f(I)_k \end{bmatrix}
$$

Then, we will form the binary string according to the following rule:
$$
s_j = \begin{cases} 1, f(I)_j > 0 \\ 0, f(I)_j \leq 0 \end{cases}
$$

Also we will denote the distance between two binary string $s=s_1s_2\dots s_k$ and $h=h_1h_2 \dots h_k$ as follows:
$$
\delta(s, h) = \frac{1}{k} \sum_{j=1}^k |s_j - h_j|
$$

The similarity of two binary string we will denote as $\sigma(s, h) = 1 - \delta(s,h)$

In [ ]:
# Forming the string batches from the feature vectors
string_batches = help_binary.form_string_batches(feature_vectors)

Using the set of pairs formed before, we can split it into the set of pairs with the same people and with different people (each of size $n_p' = n_p/2$). To get the average similarity between two same or different people, we will simple average the similarity function applied to each pair in the corresponding set, that is:
$$
\sigma_{\text{same/different}} = \frac{1}{n_p'}\sum_{t=1}^{n_p'} \sigma(I_{i_1^{[t]}}^{(n_1^{[t]})}, I_{i_2^{[t]}}^{(n_2^{[t]})})
$$

In [ ]:
same_pairs, different_pairs = help_shaper.split_pairs(pairs)

print('Average similarity between same person', help_binary.get_average_similarity(string_batches, same_pairs))
print('Average similarity between different people', help_binary.get_average_similarity(string_batches, different_pairs))